In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Master Thesis/data
%ls

Mounted at /content/drive
/content/drive/MyDrive/Master Thesis/data
 aggregatedForAllComps_esg.csv                marketCap_processedFinal.csv
 aggregatedForAllComps_esgRefinitiv.csv       plotAcf_ClimateBert_M.png
 aggregatedForAllComps_RepRiskRating.csv      plotAcf_TwitterBert_M.png
 aggregatedForAllComps_RepRiskRRI.csv         predCmmtsAgg_M.csv
 aggregatedForAllControls_esg.csv             predCmmtsAgg_Y.csv
 aggregatedForAllControls_esgRefinitiv.csv    ProcessedComps/
 aggregatedForAllControls_RepRiskRating.csv   Results/
 aggregatedForAllControls_RepRiskRRI.csv      ResultsTest/
 allControls.csv                              sp500Monthlyrets.csv
 controls.csv                                'sp500Sectors (1).csv'
 ESG/                                        'To be Checked but already processed'/
 marketCap.csv


In [ ]:
import glob
import pandas as pd
import numpy as np
from datetime import datetime

import datetime as dt
import calendar

from tqdm.notebook import tqdm

In [ ]:
instaCompanies = glob.glob('./ProcessedComps/*.csv')
compList = []
for i in instaCompanies:
  compList.append(i[17:-9])

compList.sort()


In [ ]:
def getResults(comp,resultName, dropCols):
  path = './Results/%s'%comp
  df = pd.read_csv(path + '/%s.csv'%resultName)
  df['PostDate'] = pd.to_datetime(df['PostDate']).dt.strftime('%Y-%m-%d')
  df['PostDate'] = pd.to_datetime(df['PostDate'])
  df.set_index(['PostDate'], inplace = True)

  df.drop(columns = dropCols, inplace = True)

  if 'Unnamed: 0.1' in df.columns:
    df.drop(columns = ['Unnamed: 0.1'], inplace = True)

  if 'Unnamed: 0' in df.columns:
    df.drop(columns = ['Unnamed: 0'], inplace = True)

  df.sort_index(inplace = True)
  return df

In [ ]:
userSentiment={}
companyClass = {}
for c in tqdm(compList):
  userSentiment[c] = getResults(c,'predCmmtsMean_neutralized', ['Description']) #get user sentiments per company
  companyClass[c] = getResults(c,'predDescript', ['Description', 'PostsComments']) #get post classification per company


  0%|          | 0/185 [00:00<?, ?it/s]

Filtering for comments ClimateBERT

In [ ]:
def aggByPersonalizedTime(diffDates,resultsDf, dropCols = None, timeInterval = 365):
  aggUserSentiment = pd.DataFrame(columns = resultsDf.columns)
  dates = []
  cnt=0
  for d in range( len ( diffDates ) ):
    if cnt == 0 and d == 0:
      dateUserS = pd.DataFrame(resultsDf[(resultsDf.index <= diffDates[d]) & (resultsDf.index >= diffDates[d] - pd.Timedelta(timeInterval,'day'))].apply('mean')).T
      if dateUserS.isna().all().all():
        continue

    else:
      dateUserS = pd.DataFrame(resultsDf[(resultsDf.index <= diffDates[d]) & (resultsDf.index > diffDates[d-1])].apply('mean')).T
      if dateUserS.isna().all().all() and cnt==0:
        continue

    if dateUserS.isna().all().all():
      aggUserSentiment = pd.concat([aggUserSentiment, aggUserSentiment.iloc[cnt-1:cnt,:]])
    else:
      aggUserSentiment = pd.concat([aggUserSentiment, dateUserS])
    cnt+=1
    dates.append(diffDates[d])

  if dropCols == None:
    aggUserSentimentFinal = aggUserSentiment

  else:
    aggUserSentimentFinal = aggUserSentiment.drop(columns = dropCols)

  aggUserSentimentFinal.index = dates
  return aggUserSentimentFinal

def getGroupedDataFromProvider(compDf,firstDate,provider,company,dateCol,dropCol, diffCol, dateFormat = '%Y-%m-%d', index = None):
  path = './Results/%s'%company

  if index==None:
    esg = pd.read_csv(path+'/%s.csv'%provider)[[dateCol]+diffCol].dropna(axis=1)
    esg[dateCol] = pd.to_datetime(esg[dateCol], format = dateFormat)
    esg.set_index(dateCol,inplace=True)
    esgDiffDf = esg[(esg!=0).any(axis=1)]

  else:
    esg = pd.read_csv(path+'/%s.csv'%provider)[diffCol]
    esg.index = index
    esg.index = pd.to_datetime(esg.index, format = '%d.%m.%Y')
    esgDiffDf = esg.sort_index().diff()

  esgDiffDf = esgDiffDf[esgDiffDf.index > firstDate]
  esgDiffDates = esgDiffDf.index.tolist()

  cols = [col for col in esgDiffDf.columns if 'Diff' in col]

  prov = aggByPersonalizedTime(esgDiffDates,compDf,dateCol,dropCol)

  if not cols:
    provCompared = pd.concat([prov, esgDiffDf],axis=1)
  else:
    provCompared = pd.concat([prov, esgDiffDf[cols]],axis=1)
  return provCompared.dropna()

#Aggregate sentiments uniformly & RRI changes uniformly

---



In [ ]:
results = pd.DataFrame()

for c in compList:
  results = pd.concat([results, userSentiment[c]])

resSorted = results.sort_index()
filtRes = resSorted[(resSorted.index >= pd.Timestamp('2016-01-01')) & (resSorted.index <= pd.Timestamp('2023-06-01'))]
finalRes = filtRes[['sentiment','sentimentGen']].groupby(level = 0).mean()

resCB_M = finalRes.resample('M').agg({'sentiment':lambda x: np.mean(x) if not x.empty else 0})
resEB_M = finalRes.resample('M').agg({'sentimentGen':lambda x: np.mean(x) if not x.empty else 0})

resCB_Y = finalRes.resample('Y').agg({'sentiment':lambda x: np.mean(x) if not x.empty else 0})
resEB_Y = finalRes.resample('Y').agg({'sentimentGen':lambda x: np.mean(x) if not x.empty else 0})


pd.concat([resCB_M,resEB_M],axis=1).to_csv('./predCmmtsAgg_M.csv')
pd.concat([resCB_Y,resEB_Y],axis=1).to_csv('./predCmmtsAgg_Y.csv')


#Aggregate over all companies in universe

---



Concat all the ESG values for every company into one df, aggregating by Mean if have same dates

In [ ]:
allEsg = {}
provs = ['RepRiskRating', 'RepRiskRRI', 'esg', 'esgRefinitiv']
dates = {'RepRiskRating':'date', 'RepRiskRRI':'date', 'esg':'Start Date', 'esgRefinitiv':'Post Date'}
diffCols = {'RepRiskRating':['ratingchange'], 'RepRiskRRI':['current_rriDiff'], 'esg':['SocialDiff',	'GovDiff',	'EnvDiff',	'ESGDiff'], 'esgRefinitiv':['ESG Score',	'ESG Controversies Score',	'ESG Combined Score',	'Environmental Pillar Score','Social Pillar Score',	'Governance Pillar Score']}
dateFormat = {'RepRiskRating':'%Y-%m-%d', 'RepRiskRRI':'%Y-%m-%d', 'esg':'%d.%m.%Y', 'esgRefinitiv':'%d.%m.%Y'}

for p in tqdm(provs):
  dateCol = dates[p]
  diffCol = diffCols[p]
  dateForm = dateFormat[p]

  df = pd.DataFrame()

  for c in compList:
    path = './Results/%s'%c
    try:
      if p != 'esgRefinitiv':
        esg = pd.read_csv(path+'/%s.csv'%p)[[dateCol]+diffCol].dropna(axis=1)
        esg[dateCol] = pd.to_datetime(esg[dateCol], format = dateForm)
        esg.set_index(dateCol,inplace=True)
        esgDiffDf = esg[(esg!=0).any(axis=1)]

      else:
          esg = pd.read_csv(path+'/%s.csv'%p)[diffCol]
          esg.index = ['31.12.2021','31.12.2020','31.12.2019','31.12.2018','31.12.2017','31.12.2016']
          esg.index = pd.to_datetime(esg.index, format = dateForm)
          esgDiffDf = esg.sort_index().diff()
          esgDiffDf.rename_axis(dateCol, inplace = True)
    except:
      continue

    df = pd.concat([df,esgDiffDf])
  df.dropna(how = 'all', inplace = True)
  allEsg[p] = df.reset_index().rename(columns = {dateCol :'PostDate'}).groupby(by='PostDate').mean().sort_index()


  0%|          | 0/4 [00:00<?, ?it/s]

Concat values for all companies together

In [ ]:
df = pd.DataFrame(columns = userSentiment['ADOBE INC'].columns)
for c in compList:
  df = pd.concat([df, userSentiment[c]])

allComps = df.reset_index().rename(columns = {'index':'PostDate'}).groupby(by='PostDate').mean().sort_index()

provCompared = {}
for p in provs:
  prov = aggByPersonalizedTime(allEsg[p].index,allComps,['env','soc','gov'])
  provCompared[p] = pd.concat([prov, allEsg[p]],axis=1)
  provCompared[p].to_csv('./aggregatedForAllComps_%s.csv'%p)

<ipython-input-54-67fba3f85317>:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  allComps = df.reset_index().rename(columns = {'index':'PostDate'}).groupby(by='PostDate').mean().sort_index()


#Get all ESG measures aggregated per Company and save (run once)

---



In [ ]:
rriCompared = {}
bbgResultsCompared = {}
refResultsCompared = {}
noRRI = []
noRef = []
for c in tqdm(compList):
  #print('-- starting %s --'%c)
  company = userSentiment[c]
  firstDate =  company.index.tolist()[0]

  try:
    #rriCompared[c] = getGroupedDataFromProvider(company, firstDate,'RepRiskRRI',c,'date',['PostIndex','env','soc','gov'],['current_rriDiff'])
    getGroupedDataFromProvider(company,firstDate,'RepRiskRating',c,'date',['PostIndex','env','soc','gov'],['ratingchange']).to_csv('./Results/%s/predCmmtsAgg_rrating.csv'%c)
    getGroupedDataFromProvider(company,firstDate,'RepRiskRRI',c,'date',['PostIndex','env','soc','gov'],['current_rriDiff']).to_csv('./Results/%s/predCmmtsAgg_rri.csv'%c)
  except:
    noRRI.append(c)

  #bbgResultsCompared[c] = getGroupedDataFromProvider(company, firstDate,'esg',c,'Start Date',['PostIndex','env','soc','gov'],['SocialDiff',	'GovDiff',	'EnvDiff',	'ESGDiff'],'%d.%m.%Y')

  getGroupedDataFromProvider(company,firstDate,'esg',c,'Start Date',['PostIndex','env','soc','gov'],['SocialDiff',	'GovDiff',	'EnvDiff',	'ESGDiff'],'%d.%m.%Y').to_csv('./Results/%s/predCmmtsAgg_bbg.csv'%c)

  try:
    #refResultsCompared[c] = getGroupedDataFromProvider(company, firstDate,'esgRefinitiv',c,'Post Date',['PostIndex','env','soc','gov'],['ESG Score',	'ESG Controversies Score',	'ESG Combined Score',	'Environmental Pillar Score','Social Pillar Score',	'Governance Pillar Score'],'%d.%m.%Y',index = ['31.12.2021','31.12.2020','31.12.2019','31.12.2018','31.12.2017','31.12.2016'] )
    getGroupedDataFromProvider(company,firstDate,'esgRefinitiv',c,'Post Date',['PostIndex','env','soc','gov'],['ESG Score',	'ESG Controversies Score',	'ESG Combined Score',	'Environmental Pillar Score','Social Pillar Score',	'Governance Pillar Score'],'%d.%m.%Y',index = ['31.12.2021','31.12.2020','31.12.2019','31.12.2018','31.12.2017','31.12.2016'] ).to_csv('./Results/%s/predCmmtsAgg_refi.csv'%c)
  except:
    noRef.append(c)


  #print('-- finished %s --'%c)

In [ ]:
noRef

['ADVANCE AUTO PARTS INC', 'GE HEALTHCARE TECHNOLOGY', 'NEWELL BRANDS INC']

In [ ]:
noRRI

['ANSYS INC',
 'GE HEALTHCARE TECHNOLOGY',
 'JACOBS SOLUTIONS INC',
 'SEAGATE TECHNOLOGY HOLDINGS',
 'SOLAREDGE TECHNOLOGIES INC']

#Aggregate Controls

get the controls per company (RUN ONCE)

In [ ]:
controls = pd.read_csv('./controls.csv')
controls['public_date'] = pd.to_datetime(controls['public_date'])

mkt = pd.read_csv('./marketCap_processedFinal.csv')
mkt['date'] = pd.to_datetime(mkt['date'])


#Get tickers
n = pd.read_csv('./ESG/SP500_Names.csv')
names = pd.DataFrame(np.vstack([n.columns, n]), columns = ['Ticker', 'Name'])
names[['Tickers', 'x', 'y']] = names['Ticker'].str.split(expand = True)
tickers = names[['Tickers','Name']].set_index('Tickers')
tickersDict = tickers.to_dict()['Name']

for k, v in tqdm(tickersDict.items()):
  #print(v)
  mktComp = mkt[mkt['TICKER'] == k][['date',' MKTC ']].rename(columns = {' MKTC ':'MKTC'}).set_index('date').resample('M').last()
  ctls =  controls[controls['TICKER']==k].set_index('public_date')

  df = pd.merge(ctls, mktComp, left_index = True, right_index = True)
  df['MKTC'] = np.log(df['MKTC'].astype('float64'))

  #save to corresponding folder
  compCtl = df.to_csv('./Results/%s/controls.csv'%v)

<ipython-input-10-1602e4faca93>:4: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  mkt = pd.read_csv('./marketCap_processedFinal.csv')


  0%|          | 0/503 [00:00<?, ?it/s]

#Merging controls with data from RepRisk

In [ ]:
def getControlledData(comp):
  ctls = pd.read_csv('./Results/%s/controls.csv'%comp)[['public_date','bm','pe_exi',	'roa',	'debt_assets'	,'curr_ratio', 'MKTC'	]]
  ctls['Date'] = pd.to_datetime(ctls['public_date'])
  ctls.set_index('Date', inplace = True)
  return ctls.drop(columns = ['public_date'])

In [ ]:
provs = ['RepRiskRating', 'RepRiskRRI']
dates = {'RepRiskRating':'date', 'RepRiskRRI':'date'}
diffCols = {'RepRiskRating':['ratingchange'], 'RepRiskRRI':['current_rriDiff']}
dateFormat = {'RepRiskRating':'%Y-%m-%d', 'RepRiskRRI':'%Y-%m-%d'}
label = {'RepRiskRating':'rrating', 'RepRiskRRI':'rri'}
noControls = []

for p in provs:

  lab = label[p]

  for c in compList:
    print(c)
    if c in noRRI:
      continue

    try:
      compDf = getControlledData(c)
      start = compDf.index.min()
      end = compDf.index.max()
    except:
      noControls.append(c)

    diff = pd.read_csv('./Results/%s/predCmmtsAgg_%s.csv'%(c,lab))
    diff['Unnamed: 0'] = pd.to_datetime(diff['Unnamed: 0'])


    #diff = diff[diff['Unnamed: 0'] >= '2016-02-01'].reset_index().drop(columns = ['index']) #>= that date because only retrieved controls from 2016-01-31

    diff =diff[(diff['Unnamed: 0'] >= max(start,pd.to_datetime('2016-02-01'))) & (diff['Unnamed: 0'] <= end) ].reset_index().drop(columns = ['index'])

    ctls = pd.DataFrame(columns = ['bm'	,'pe_exi'	,'roa'	,'debt_assets'	,'curr_ratio', 'MKTC'])
    for d in diff['Unnamed: 0']:

      if d.month == 1:
        year = d.year - 1
        month = 12

      else:
        month = d.month - 1
        year = d.year

      lastData = compDf.loc['%s-%s'%(year,month)]

      if lastData.isna().all().all():
        #ctls = pd.concat([ctls, pd.DataFrame([[np.nan]*6], columns = ctls.columns, index = [pd.to_datetime('%s-%s-%s'%(year, month, calendar.monthrange(year, month)[1]))])])
        ctls = pd.concat([ctls,pd.DataFrame([ctls.iloc[-1,:]])])

      else:
        ctls = pd.concat([ctls, lastData])

    ctls.reset_index(inplace = True)
    pd.merge(diff,ctls,left_index = True, right_index = True).set_index('Unnamed: 0').to_csv('./Results/%s/predCmmtsAgg_%sControlledv3.csv'%(c,lab))


#Aggregate controls from all companies

Run once to save

In [ ]:
cols = pd.read_csv('./Results/APPLE INC/controls.csv').columns
allCtls = pd.DataFrame(columns = cols)

for c in tqdm(compList):
  allCtls = pd.concat([allCtls,pd.read_csv('./Results/%s/controls.csv'%c)])

allCtls.drop(columns = ['Unnamed: 0','date', 'TICKER', 'permno'], inplace = True)
allCtls.set_index('public_date', inplace = True)

allCtls.groupby(by='public_date').mean().to_csv('./allControls.csv')

  0%|          | 0/185 [00:00<?, ?it/s]



---



#Get controls for all companies aggregated by provider (just take value from last month)

In [ ]:
allControls = pd.read_csv('./allControls.csv').set_index('public_date')
#allControls['MKTC'] = np.log(allControls['MKTC'])
allControls.index = pd.to_datetime(allControls.index)
cols = allControls.columns

start = allControls.index.min() #get first and last date for controls, should be 2016-01 to 2022-12
end = allControls.index.max()

provCompared = {}
provs = ['RepRiskRating', 'RepRiskRRI', 'esg', 'esgRefinitiv']
for p in tqdm(provs):

  df = pd.read_csv('./aggregatedForAllComps_%s.csv'%p).set_index('Unnamed: 0')
  df.index = pd.to_datetime(df.index)
  indepVar =df[(df.index <= end ) & (df.index >= max(start, pd.to_datetime('2016-02-29')) )] #start either Feb 29 2016 or later date, this is because earliest controls is Jan 31 2016
  #indepVar = allEsg[p][( allEsg[p].index <= end ) & ( allEsg[p].index >= max(start, pd.to_datetime('2016-02-29')) )] #start either Feb 29 2016 or later date, this is because earliest controls is Jan 31 2016
  dates = indepVar.index

  ctls = pd.DataFrame(columns = cols)

  for d in dates:

    if d.month == 1:
      year = d.year - 1
      month = 12

    else:
      month = d.month - 1
      year = d.year

    lastData = allControls.loc['%s-%s'%(year,month)]

    if lastData.isna().all().all():
      ctls = pd.concat([ctls,pd.DataFrame([ctls.iloc[-1,:]])])

    else:
      ctls = pd.concat([ctls, lastData])

  ctls.reset_index(inplace = True)
  indepVar.reset_index(inplace = True)
  pd.merge(ctls, indepVar,left_index = True, right_index = True).set_index('Unnamed: 0').to_csv('./aggregatedForAllControls_%s.csv'%p)

  #pd.merge(diff,ctls,left_index = True, right_index = True).set_index('Unnamed: 0').to_csv('./Results/%s/predCmmtsAgg_%sControlledv2.csv'%(c,lab))


  0%|          | 0/4 [00:00<?, ?it/s]